<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Web-Scrap---Transfers" data-toc-modified-id="Web-Scrap---Transfers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Web Scrap - Transfers</a></span></li><li><span><a href="#Web-Scrap---Player-Agents" data-toc-modified-id="Web-Scrap---Player-Agents-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Web Scrap - Player Agents</a></span></li><li><span><a href="#Web-Scrap---Manager-Agents" data-toc-modified-id="Web-Scrap---Manager-Agents-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Web Scrap - Manager Agents</a></span></li><li><span><a href="#Merge-data" data-toc-modified-id="Merge-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Merge data</a></span></li><li><span><a href="#Clean-data" data-toc-modified-id="Clean-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Clean data</a></span><ul class="toc-item"><li><span><a href="#Create-Transfer-type" data-toc-modified-id="Create-Transfer-type-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Create Transfer type</a></span></li><li><span><a href="#Transfer-fee---integer-amount" data-toc-modified-id="Transfer-fee---integer-amount-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Transfer fee - integer amount</a></span></li></ul></li><li><span><a href="#Build-Network" data-toc-modified-id="Build-Network-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Build Network</a></span><ul class="toc-item"><li><span><a href="#Nodes" data-toc-modified-id="Nodes-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Nodes</a></span></li><li><span><a href="#Edges" data-toc-modified-id="Edges-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Edges</a></span></li><li><span><a href="#Network" data-toc-modified-id="Network-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Network</a></span></li><li><span><a href="#Export" data-toc-modified-id="Export-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Export</a></span></li></ul></li></ul></div>

# Imports

In [125]:
# Beautiful Soup for web scrapping data
from bs4 import BeautifulSoup
import requests

# Save into csv file
import csv

# Pandas to reimport csv data
import pandas as pd

# Numpy for array manipulation
import numpy as np

# NetworkX
import networkx as nx

Use it for progess bar (https://github.com/alexanderkuk/log-progress)

In [2]:
def log_progress(sequence, every=None, size=None, name='Items', delete=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )
        if delete:
            box.close()

# Web Scrap - Transfers

In [3]:
transfer_infos = ["Player Name", "Player Link", "Player position", "From club", "To club", "From competition", "To competition", "From manager", "From manager link", "To manager", "To manager link", "Market value", "Fee", "Age"]

In [4]:
base_url = "https://www.transfermarkt.com"
transfer_url = base_url + '/transfers/transfertagedetail/statistik/top/land_id_zu/0/land_id_ab/0/leihe//datum/{0}/page/{1}'

In [5]:
headers = {'User-Agent': 'Mozilla/5.00'}

In [6]:
i=0
nbr_failure = 0


YearRange = range(2015,2017) # 2015 - 2016
Months = range(1,13)
DaysInMonths = [0,31,28,31,30,31,30,31,31,30,31,30,31]


for year in YearRange:
    for month in log_progress(Months, name="Month"):
        for day in log_progress(range(1,DaysInMonths[month]+1), name="Day", delete=True):
            date = str(year)+'-'+str(month)+'-'+str(day)

            # create file handle
            csvfile = open('data/data-transfer-day/data-test'+date+'.csv', 'w')
            
            url = transfer_url.format(date, 1)
            r  = requests.get(url, headers=headers)
            tranfer_data_page = BeautifulSoup(r.text, 'html.parser')
            
            try:
                nbr_pages = int(tranfer_data_page.find('li', attrs={"class":"letzte-seite"}).find('a')['href'].split('page/')[1])
            except:
                nbr_pages = 1

            try:
                for page in log_progress(range(1,nbr_pages+1), every=1, name="Date: "+date+"    - Page", delete=True): #Progress bar for each page for a given day
                    url = transfer_url.format(date, page)
                    r  = requests.get(url, headers=headers)
                    tranfer_data_page = BeautifulSoup(r.text, 'html.parser')


                    links = tranfer_data_page.find_all('a', href=True)
                    for a in log_progress([l for l in links if "/jumplist/transfers/" in l['href']], every=1, name="Page "+str(page)+"   - Player", delete=True): #Progress bar for each transfer in a page
                        i += 1

                        try:
                            # Player page - Transfer details
                            p_transfer_url = base_url + a['href']
                            r2  = requests.get(p_transfer_url, headers=headers)
                            soup = BeautifulSoup(r2.text, 'html.parser')

                            # Player info
                            player_link = soup.find("li", attrs={"id":"Profile"}).find("a")['href']
                            player_name = player_link.split('/')[1]
                            player_position = soup.find("span", text="Position:").find_next().get_text()
                            player_position = player_position.replace("\r", "").replace("\t", "").replace("\n", "")

                            # Clubs
                            clubs = soup.find("td", attrs={"class":"zentriert hauptlink no-border-rechts no-border-links"})
                            from_club = clubs.find_previous('td').find_all('a')[-1].text
                            to_club   = clubs.find_next('td').find_all('a')[-1].text

                            # Competition
                            competitions = soup.find("td", text="Competition")
                            from_competition = competitions.find_previous('td').find_all('a')[-1].text
                            to_competition   = competitions.find_next('td').find_all('a')[-1].text

                            # Managers
                            managers = soup.find("td", text="Manager(s)")
                            from_manager = managers.find_previous('td').find_all('a')[-1].text
                            from_manager_link = managers.find_previous('td').find('a')['href']
                            to_manager   = managers.find_next('td').find_all('a')[-1].text
                            to_manager_link = managers.find_next('td').find('a')['href']

                            # Market Value
                            market_value = soup.find("b", text="Market value at time of change").find_parent().text.split("Market value at time of change")[-1]

                            # Fee
                            fee = soup.find("b", text="Transfer fee").find_next().find_next().text

                            # Age
                            age = soup.find("b", text="Age at the time of the transfer").find_parent().text.split("Age at the time of the transfer")[-1]


                            # Write to CSV file
                            csvfile.write("|".join([player_name, player_link, player_position, from_club, to_club, from_competition, to_competition, from_manager, from_manager_link, to_manager, to_manager_link, market_value, fee, age])+"\n")

                        except:
                            nbr_failure += 1
            except:
                print("Error with", page)
                pass


            # Close CSV file
            csvfile.close()

Error with 147
Error with 1
Error with 2


ConnectionError: HTTPSConnectionPool(host='www.transfermarkt.com', port=443): Max retries exceeded with url: /transfers/transfertagedetail/statistik/top/land_id_zu/0/land_id_ab/0/leihe//datum/2016-7-16/page/1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f7f28c4ab70>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [ ]:
# Merge everythin
!cat data/data-transfer-day/*.csv > data/data-transfer-day/entire_data.csv

# Web Scrap - Player Agents

In [ ]:
df = pd.read_csv("data/data-transfer-day/entire_data.csv", sep='|', names=transfer_infos)
df.head(2)

In [ ]:
df.describe()

In [ ]:
df['Player Agent'] = None

for pl in log_progress(df['Player Link'].unique()):
    r = requests.get(base_url+pl, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        agent = soup.find('th', text="Player agents:").find_next().text.replace("\n","")
        df.loc[df['Player Link'] == pl,'Player Agent'] = agent
    except:
        pass

# Web Scrap - Manager Agents

In [ ]:
fml = df['From manager link'].unique()
print("Nbr of 'From managers':    ",len(fml))
      
tml = df['To manager link'].unique()
print("Nbr of 'To managers':      ",len(tml))

print("Nbr of unique mangers:     ", len(list(set(np.concatenate((fml,tml))))))

In [ ]:
df_manager = pd.DataFrame(list(set(np.concatenate((fml,tml)))), columns=["ManagerLink"])

df['From manager agent'] = None
df['To manager agent'] = None


for pl in log_progress(df_manager['ManagerLink'].unique(), every=1):
    r = requests.get(base_url+pl, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        agent = soup.find('span', text="Agents:").find_next().text.replace("\n","")
        df.loc[df['From manager link'] == pl,'From manager agent'] = agent
        df.loc[df['To manager link'] == pl,'To manager agent'] = agent
    except:
        pass

# Merge data

In [ ]:
df.to_csv("data/data.csv")

# Clean data

In [98]:
df = pd.read_csv("data/data.csv", index_col=0)
df.sample(5)

,Player Name,Player Link,Player position,From club,To club,From competition,To competition,From manager,From manager link,To manager,To manager link,Market value,Fee,Age,From manager agent,To manager agent,Player Agent
30458,bruno-moreira,/bruno-moreira/profil/spieler/56183,Centre-Forward,Paços Ferreira,Buriram Utd.,Primeira Liga,Thai Premier League,Carlos Pinto,/carlos-pinto/profil/trainer/31663,Afshin Ghotbi,/afshin-ghotbi/profil/trainer/10324,"2,75 Mill. €",Free transfer,28 years 09 months 25 days,Footconect Fma,NaN,Soccer ProMaster
15165,william-troost-ekong,/william-troost-ekong/profil/spieler/192875,Centre-Back,FC Groningen,KAA Gent,Eredivisie,Jupiler Pro League,Erwin van de Looi,/erwin-van-de-looi/profil/trainer/12458,Hein Vanhaezebrouck,/hein-vanhaezebrouck/profil/trainer/14472,300 Th. €,Free transfer,21 years 10 months 25 days,NaN,NaN,GBS Group (International)
14510,kanu,/kanu/profil/spieler/86057,Centre-Back,Vit. Guimarães,EC Vitória,Primeira Liga,Campeonato Brasileiro Série B,Armando Evangelista,/armando-evangelista/profil/trainer/28508,Vágner Mancini,/vagner-mancini/profil/trainer/5667,"1,40 Mill. €",Loan,31 years 02 months 18 days,NaN,NaN,NaN
12939,ventsislav-hristov,/ventsislav-hristov/profil/spieler/156556,Centre-Forward,HNK Rijeka,KF Skënderbeu,1.HNL,Kategoria Superiore,Matjaz Kek,/matjaz-kek/profil/trainer/1519,Gentian Lici,/gentian-lici/profil/trainer/51904,700 Th. €,Loan,26 years 08 months 01 day,NaN,NaN,Global Management Agency
26114,salim-kerkar,/salim-kerkar/profil/spieler/98338,Left Midfield,PFC Beroe Stara Zagora,FK Vereya,A Grupa - Relegation group,A Grupa - Relegation group,Plamen Lipenski,/plamen-lipenski/profil/trainer/46469,Aleksandar Tomash,/aleksandar-tomash/profil/trainer/24655,500 Th. €,Free transfer,29 years 02 months 16 days,NaN,NaN,no agent


In [99]:
df.describe()

,Player Name,Player Link,Player position,From club,To club,From competition,To competition,From manager,From manager link,To manager,To manager link,Market value,Fee,Age,From manager agent,To manager agent,Player Agent
count,44778,44778,44778,44778,44778,44778,44778,44778,44778,44778,44778,44778,44767,44778,6046,5469,22993
unique,31221,31793,16,3600,3474,327,316,5210,5220,5104,5113,141,613,7066,244,233,2015
top,paulinho,/jonny-margetts/profil/spieler/364324,Centre-Forward,Juventus,Monza,Serie A,Primera División,Massimiliano Allegri,/massimiliano-allegri/profil/trainer/7671,Hüseyin Kalpar,/huseyin-kalpar/profil/trainer/12053,-,Free transfer,23 years 03 months 13 days,no agent,no agent,no agent
freq,18,6,7653,151,55,1599,1121,151,151,50,50,7234,19743,26,593,493,2275


## Create Transfer type

In [101]:
transfer = ['Mill. €', 'Th. €', '1 €']
loan = ['Loan fee:', 'Loan']
free = ['gratuito', 'Gratuito', 'free transfer', 'Draft', 'draft', 'Free transfer', 'nan', 'Libre para traspaso', '-', '?', '0', 'free', 'frei', 'svincolato', 'bez odstępnego', 'a']
swap = ['Swap deal', 'Trade', 'trade', 'Tausch', 'Spielertausch']

In [102]:
df['Transfer Type'] = None

In [110]:
df['Transfer Type'] = None

for tranfer_type in ["loan", "swap", "transfer", "free"]:
    df2 = df['Transfer Type'].isnull()
    for fee in log_progress(df[df2]['Fee'].unique(), every=1):
        if str(fee)=='nan': continue

        for type_fee in eval(tranfer_type):
            if type_fee in fee:
                df.loc[df['Fee']==fee, 'Transfer Type'] = tranfer_type

In [111]:
df['Transfer Type'].value_counts()

free        30994
loan        10773
transfer     2913
swap           87
Name: Transfer Type, dtype: int64

In [112]:
df[df.Fee.isnull()]

,Player Name,Player Link,Player position,From club,To club,From competition,To competition,From manager,From manager link,To manager,To manager link,Market value,Fee,Age,From manager agent,To manager agent,Player Agent,Transfer Type
2572,afran-izmailov,/afran-izmailov/profil/spieler/121842,Right Wing,Xäzär Länkäran,Inter Baku,Premyer Liqasi,Premyer Liqasi,Elbeus Mammadov,/elbeus-mammadov/profil/trainer/38668,Kakhaber Tskhadadze,/kakhaber-tskhadadze/profil/trainer/10419,400 Th. €,NaN,26 years 02 months 25 days,NaN,NaN,Cosmosport,None
7081,aly-hassan,/aly-hassan/profil/spieler/220993,Centre-Forward,Fort Lauderdale,Ottawa Fury,NASL Fall Championship,NASL Fall Championship,Iván Guerrero,/ivan-guerrero/profil/trainer/40766,Marc Dos Santos,/marc-dos-santos/profil/trainer/13259,50 Th. €,NaN,26 years 01 month 11 days,NaN,Gold World Stars,NaN,None
12780,alexander-langen,/alexander-langen/profil/spieler/303311,Right-Back,TSV 1860 U17,Ingolstadt U19,B-Jgd. BL Süd/Südwest,A-Jgd. BL Süd/Südwest,Josef Albersinger,/josef-albersinger/profil/trainer/9904,Roberto Pätzold,/roberto-patzold/profil/trainer/35740,-,NaN,17 years 05 months 02 days,NaN,NaN,BoaVista Consulting GmbH,None
24305,dejan-gavric,/dejan-gavric/profil/spieler/330108,Right Midfield,FK Derventa,FK Krupa,Prva Liga RS,Prva Liga RS,Bojan Magazin,/bojan-magazin/profil/trainer/47459,Slobodan Starcevic,/slobodan-starcevic/profil/trainer/25027,25 Th. €,NaN,18 years 06 months 03 days,NaN,NaN,NaN,None
26536,dae-ho-kim,/dae-ho-kim/profil/spieler/134743,Keeper,Asan Mugunghwa,Jeonnam Dragons,K-League Challenge,K-League Classic,Heung-sil Lee,/heung-sil-lee/profil/trainer/23161,Sang-rae No,/sang-rae-no/profil/trainer/37358,150 Th. €,NaN,30 years 08 months 16 days,NaN,NaN,NaN,None
27310,sinan-jakupovic,/sinan-jakupovic/profil/spieler/284989,Right Midfield,HNK Capljina,Velez Mostar,Prva liga FBiH,Premijer Liga,Damir Borovac,/damir-borovac/profil/trainer/29604,Zijo Tojaga,/zijo-tojaga/profil/trainer/45696,50 Th. €,NaN,21 years 05 months 05 days,NaN,NaN,no agent,None
28235,kevin-oliveira,/kevin-oliveira/profil/spieler/312744,Attacking Midfield,Benfica B,Swope Park,Segunda Liga,USL Pro,Hélder Cristóvão,/helder-cristovao/profil/trainer/11055,Marc Dos Santos,/marc-dos-santos/profil/trainer/13259,-,NaN,19 years 07 months 26 days,NaN,Gold World Stars,Soccer Features Limited,None
28555,seong-nam-ahn,/seong-nam-ahn/profil/spieler/113532,Attacking Midfield,Gwangju FC,Gyeongnam FC,K-League Classic,K-League Challenge,Ki-Il Nam,/ki-il-nam/profil/trainer/34529,Jong-bu Kim,/jong-bu-kim/profil/trainer/54825,600 Th. €,NaN,31 years 10 months 13 days,NaN,NaN,NaN,None
36912,chang-kyun-im,/chang-kyun-im/profil/spieler/263882,Attacking Midfield,Gyeongnam FC,Suwon FC,K-League Challenge,K-League Classic,Jong-bu Kim,/jong-bu-kim/profil/trainer/54825,Deok-Je Cho,/deok-je-cho/profil/trainer/28144,50 Th. €,NaN,26 years 02 months 29 days,NaN,NaN,NaN,None
38465,harallamb-qaqi,/harallamb-qaqi/profil/spieler/189690,Centre-Back,Hellas Verona,KF Laçi,Serie B,Kategoria Superiore,Fabio Pecchia,/fabio-pecchia/profil/trainer/19635,Marcello Troisi,/marcello-troisi/profil/trainer/41047,25 Th. €,NaN,22 years 10 months 10 days,NaN,NaN,NaN,None


## Transfer fee - integer amount

In [121]:
df["Transfer Value"] = 0

for fee in df[df['Transfer Type'] == 'transfer']['Fee'].unique():
    if " Mill" in fee:
        value = float(fee.split(" Mill")[0].replace(',','.'))*1000000
    
    if " Th." in fee:
        value = float(fee.split(" Th")[0].replace(',','.'))*1000
    
    if "1 €" == fee:
        value = 1
        
    df.loc[df['Fee']==fee, 'Transfer Value'] = value

# Build Network

In [232]:
G = nx.MultiGraph()

> We are using **Multigraph**: Is it a problem ?

## Nodes

Nodes are clubs

In [233]:
fc = df["From club"].unique()
tc = df["To club"].unique()
all_clubs = list(set(np.concatenate((fc,tc))))

In [234]:
for c in log_progress(all_clubs, every=1):
    clubs = df[df["From club"]==c]
    
    if len(clubs)==0:
        clubs = df[df["To club"]==c]
        competition = clubs[:1]["To competition"].values[0]
    else:
        competition = clubs[:1]["From competition"].values[0]
    
    G.add_node(c, competition=competition)

## Edges

In [235]:
for tranfer in log_progress(df.iterrows(), every=10):
    data = tranfer[1]
    from_club = data["From club"]
    to_club = data["To club"]
    
    player = data["Player Name"]
    player_position = data["Player position"]
    player_age = data["Age"]
    player_agent = data["Player Agent"]
    
    transfer_type = data["Transfer Type"]
    transfer_value = data["Transfer Value"]
    
    from_manager = data["From manager"]
    from_manager_agent = data["From manager agent"]
    to_manager = data["To manager"]
    to_manager_agent = data["To manager agent"]
    
    G.add_edge(from_club, str(to_club),
                  player = str(player),
                  playerPosition = str(player_position),
                  playerAge = str(player_age),
                  transferType = str(transfer_type),
                  transferValue = str(transfer_value),
                  fromManager = str(from_manager),
                  fromManagerAgent = str(from_manager_agent),
                  toManager = str(to_manager),
                  toManagerAgent = str(to_manager_agent)
              )

## Network

In [238]:
print(nx.info(G))

Name: 
Type: MultiGraph
Number of nodes: 3904
Number of edges: 44778
Average degree:  22.9395


## Export

In [239]:
nx.write_gml(G, "all_transfers_network.gml")